In [ ]:
pip install findspark

In [ ]:
pip install requests

In [ ]:
import json
import requests
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import concat, concat_ws, col, initcap, lower, substring, lit

In [ ]:
spark = SparkSession.builder.master("local[*]").appName('CreditCardData').getOrCreate()

In [ ]:
from pyspark.sql.types import StructType,StructField, StringType,IntegerType,BooleanType,DoubleType,LongType
schema = StructType([
    StructField('APT_NO', IntegerType(), True),
    StructField('CREDIT_CARD_NO',LongType(),True),
    StructField('CUST_CITY',StringType(),True),
    StructField('CUST_COUNTRY',StringType(),True),
    StructField('CUST_EMAIL',StringType(),True),
    StructField('CUST_PHONE',StringType(),True),
    StructField('CUST_STATE',StringType(),True),
    StructField('CUST_ZIP',LongType(),True),
    StructField('FIRST_NAME', StringType(),True),
    StructField('LAST_NAME',StringType(), True),
    StructField('LAST_UPDATED', StringType(),True),
    StructField('MIDDLE_NAME', StringType(),True),
    StructField('SSN', LongType(), True),
    StructField('STREET_NAME',StringType(),True),
])
df_with_schema = spark.read.schema(schema).json('Credit Card Dataset\cdw_sapp_custmer.json')
df_with_schema.printSchema()


In [134]:
cust_df = spark.read.json('Credit Card Dataset\cdw_sapp_custmer.json')
#cust_df.printSchema() #print original schema for the Dataframe



cust_df.createOrReplaceTempView('CDW_SAPP_CUSTOMER') #create a temp view of the customer table
new_cust_df = spark.sql('SELECT *, CONCAT(CDW_SAPP_CUSTOMER.STREET_NAME,",",CDW_SAPP_CUSTOMER.APT_NO) as STREET_NAME_APT_NO, lower(CDW_SAPP_CUSTOMER.MIDDLE_NAME) as middle_name,  concat("(", substring(CDW_SAPP_CUSTOMER.CUST_PHONE, 1, 3), ")",\
               substring(CDW_SAPP_CUSTOMER.CUST_PHONE, 4, 3), "-",\
               substring(CDW_SAPP_CUSTOMER.CUST_PHONE, 7, 4), "000") AS FORMAT_CUST_PHONE FROM CDW_SAPP_CUSTOMER ') #query concatenates the street name and apt no, lower case for middle name, and reformats the customer phone
# reformat.printSchema()
new_cust_df.show()

type(new_cust_df)
# cust_phone = spark.sql('SELECT CONCAT("(", SUBSTRING(CUST_PHONE, 1, 3), ")", SUBSTRING(CUST_PHONE, 4, 3), "-", SUBSTRING(CUST_PHONE, 7, 4), "000") AS CUST_PHONE FROM  CDW_SAPP_CUSTOMER') #gets customers phone number
# #cust_phone.printSchema()
# #cust_phone.show()


# mid_name = spark.sql('SELECT *, LOWER(MIDDLE_NAME) AS MIDDLE_NAME FROM CDW_SAPP_CUSTOMER') #this query obtains the customers middle name and places it into a lower case format
# mid_name.show()

# updated_df = mid_name.select('SSN','FIRST_NAME','MIDDLE_NAME','LAST_NAME','CREDIT_CARD_NO','STREET_NAME_APT_NO','CUST_CITY','CUST_STATE','CUST_COUNTRY','CUST_ZIP','CUST_PHONE','CUST_EMAIL','LAST_UPDATED')
# #updated_address.printSchema()
# updated_df.show()

# # new_cust_df = mid_name.select('SSN','FIRST_NAME','MIDDLE_NAME','LAST_NAME','CREDIT_CARD_NO','STREET_NAME_APT_NO','CUST_CITY','CUST_STATE','CUST_COUNTRY','CUST_ZIP','CUST_PHONE','CUST_EMAIL','LAST_UPDATED')
# # updated_cust_df.printSchema()
# # new_cust_df.show()




+------+----------------+------------+-------------+--------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+--------------------+-----------+-----------------+
|APT_NO|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|          CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME|  STREET_NAME_APT_NO|middle_name|FORMAT_CUST_PHONE|
+------+----------------+------------+-------------+--------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+--------------------+-----------+-----------------+
|   656|4210653310061055|     Natchez|United States| AHooper@example.com|   1237818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         Wm|123456100|Main Street North|Main Street North...|         wm|    (123)781-8000|
|   829|4210653310102868|Wethersfield|United Sta

pyspark.sql.dataframe.DataFrame

In [133]:
cc_df = spark.read.json('Credit Card Dataset\cdw_sapp_credit.json')

#cc_df.printSchema()
cc_df.createOrReplaceTempView('CDW_SAPP_CREDIT_CARD')
date = spark.sql('SELECT *, CONCAT(DAY,"-",MONTH,"-",YEAR) AS DAY_MONTH_YEAR FROM CDW_SAPP_CREDIT_CARD')
updated_cc_df = date.select('CREDIT_CARD_NO','DAY_MONTH_YEAR','CUST_SSN','BRANCH_CODE','TRANSACTION_TYPE','TRANSACTION_VALUE','TRANSACTION_ID')

updated_cc_df.show()
type(updated_cc_df)

+----------------+--------------+---------+-----------+----------------+-----------------+--------------+
|  CREDIT_CARD_NO|DAY_MONTH_YEAR| CUST_SSN|BRANCH_CODE|TRANSACTION_TYPE|TRANSACTION_VALUE|TRANSACTION_ID|
+----------------+--------------+---------+-----------+----------------+-----------------+--------------+
|4210653349028689|     14-2-2018|123459988|        114|       Education|             78.9|             1|
|4210653349028689|     20-3-2018|123459988|         35|   Entertainment|            14.24|             2|
|4210653349028689|      8-7-2018|123459988|        160|         Grocery|             56.7|             3|
|4210653349028689|     19-4-2018|123459988|        114|   Entertainment|            59.73|             4|
|4210653349028689|    10-10-2018|123459988|         93|             Gas|             3.59|             5|
|4210653349028689|     28-5-2018|123459988|        164|       Education|             6.89|             6|
|4210653349028689|     19-5-2018|123459988|   

pyspark.sql.dataframe.DataFrame

In [145]:

branch_df = spark.read.json('Credit Card Dataset\cdw_sapp_branch.json')
branch_df.createOrReplaceTempView('CDW_SAPP_BRANCH')


branch_query = spark.sql('select *,  concat("(", substring(CDW_SAPP_BRANCH.BRANCH_PHONE, 1, 3), ")",\
               substring(CDW_SAPP_BRANCH.BRANCH_PHONE, 4, 3), "-",\
               substring(CDW_SAPP_BRANCH.BRANCH_PHONE, 7, 4)) AS BRANCH_PHONE, CDW_SAPP_BRANCH.BRANCH_ZIP,\
        CASE WHEN CDW_SAPP_BRANCH.BRANCH_ZIP IS NULL THEN "99999" ELSE CDW_SAPP_BRANCH.BRANCH_ZIP END AS BRANCH_ZIP \
        from CDW_SAPP_BRANCH')
#branch_query.show()






In [150]:
new_brach_df = branch_query.select('BRANCH_CITY','BRANCH_CODE','BRANCH_NAME',branch_query["CDW_SAPP_BRANCH.BRANCH_PHONE"],'BRANCH_STATE','BRANCH_STREET',branch_query['CDW_SAPP_BRANCH.BRANCH_ZIP'],'LAST_UPDATED')
new_brach_df.show()

+-----------------+-----------+------------+------------+------------+-------------------+----------+--------------------+
|      BRANCH_CITY|BRANCH_CODE| BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|      BRANCH_STREET|BRANCH_ZIP|        LAST_UPDATED|
+-----------------+-----------+------------+------------+------------+-------------------+----------+--------------------+
|        Lakeville|          1|Example Bank|  1234565276|          MN|       Bridle Court|     55044|2018-04-18T16:51:...|
|          Huntley|          2|Example Bank|  1234618993|          IL|  Washington Street|     60142|2018-04-18T16:51:...|
|SouthRichmondHill|          3|Example Bank|  1234985926|          NY|      Warren Street|     11419|2018-04-18T16:51:...|
|       Middleburg|          4|Example Bank|  1234663064|          FL|   Cleveland Street|     32068|2018-04-18T16:51:...|
|    KingOfPrussia|          5|Example Bank|  1234849701|          PA|        14th Street|     19406|2018-04-18T16:51:...|
|         Paters

In [151]:
spark.stop()

In [ ]:
new_cust_df.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
.option("dbtable", "creditcard_capstone.CDW_SAPP_CUSTOMER") \
.option("user", "root") \
.option("password", "password") \
.save()

In [131]:
new_branch_df.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
.option("dbtable", "creditcard_capstone.CDW_SAPP_BRANCH") \
.option("user", "root") \
.option("password", "password") \
.save()

Py4JJavaError: An error occurred while calling o1972.save.
: java.sql.SQLSyntaxErrorException: Duplicate column name 'BRANCH_ZIP'
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:120)
	at com.mysql.cj.jdbc.exceptions.SQLExceptionsMapping.translateException(SQLExceptionsMapping.java:122)
	at com.mysql.cj.jdbc.StatementImpl.executeUpdateInternal(StatementImpl.java:1333)
	at com.mysql.cj.jdbc.StatementImpl.executeLargeUpdate(StatementImpl.java:2106)
	at com.mysql.cj.jdbc.StatementImpl.executeUpdate(StatementImpl.java:1243)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.executeStatement(JdbcUtils.scala:1083)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.createTable(JdbcUtils.scala:913)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:81)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:47)
	at org.

In [136]:
new_cust_df.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
.option("dbtable", "creditcard_capstone.CDW_SAPP_CUSTOMER") \
.option("user", "root") \
.option("password", "password") \
.save()

Py4JJavaError: An error occurred while calling o2017.save.
: java.sql.SQLSyntaxErrorException: Duplicate column name 'middle_name'
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:120)
	at com.mysql.cj.jdbc.exceptions.SQLExceptionsMapping.translateException(SQLExceptionsMapping.java:122)
	at com.mysql.cj.jdbc.StatementImpl.executeUpdateInternal(StatementImpl.java:1333)
	at com.mysql.cj.jdbc.StatementImpl.executeLargeUpdate(StatementImpl.java:2106)
	at com.mysql.cj.jdbc.StatementImpl.executeUpdate(StatementImpl.java:1243)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.executeStatement(JdbcUtils.scala:1083)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.createTable(JdbcUtils.scala:913)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:81)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:47)
	at org